In [12]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def perform_stratified_sampling(sample_fraction, input_file, output_file, min_per_group=1, stratify_col='tier', random_state=42):
    """
    계층적 샘플링 수행 (기본: 티어별 샘플링).
    - sample_fraction: 각 그룹에서 비율로 뽑을 비율
    - min_per_group: 그룹이 작을 때 최소 보장 샘플 수
    - stratify_col: 계층화에 사용할 컬럼 이름
    """
    print('📂 데이터 로딩 중...')
    df = pd.read_csv(input_file)

    if stratify_col not in df.columns:
        raise ValueError(f"'{stratify_col}' 컬럼이 입력 파일에 없습니다.")

    # 그룹별로 샘플 수 계산
    def _sample_group(g):
        grp_size = len(g)
        n = max(0, int(round(grp_size * sample_fraction)))
        if n < min_per_group and grp_size > 0:
            n = min(min_per_group, grp_size)
        if n <= 0:
            return g.iloc[0:0]
        return g.sample(n=n, random_state=random_state)

    sampled_df = df.groupby(stratify_col, group_keys=False).apply(_sample_group).reset_index(drop=True)

    # 결과 출력 및 저장
    print('\n✅ 샘플링 완료!')
    print(f"전체 데이터 수: {len(df):,} 명")
    print(f"샘플링 데이터 수: {len(sampled_df):,} 명")
    print(f"샘플 내 {stratify_col} 비중:\n{(sampled_df[stratify_col].value_counts(normalize=True) * 100).round(2)}%")

    sampled_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"💾 샘플 데이터가 {output_file}에 저장되었습니다.")

In [31]:
file_path = r'D:\바탕화면\데이터 분석\프로젝트\04 최종 프로젝트'
name = 'tr1_puuids_iron_diamond'
sampling = 0.5  # 0.5%
perform_stratified_sampling(
    sample_fraction = sampling / 100,
    input_file=f'{file_path}{name}.csv', 
    output_file=f'{file_path}{sampling}%_{name}.csv'
)

📂 데이터 로딩 중...


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\바탕화면\\데이터 분석\\프로젝트\\04 최종 프로젝트tr1_puuids_iron_diamond.csv'

In [32]:
import os

# 1. 경로 설정 (맨 앞에 r을 붙이고, 맨 뒤의 슬래시는 제거했습니다)
file_path = r'D:\바탕화면\데이터 분석\프로젝트\04 최종 프로젝트'
name = 'tr1_puuids_iron_diamond'
sampling = 0.5  # 0.5%

# 2. 경로와 파일명을 안전하게 결합 (f-string 대신 os.path.join 사용)
# 함수에 전달할 최종 경로를 미리 만듭니다.
input_full_path = os.path.join(file_path, f'{name}.csv')
output_full_path = os.path.join(file_path, f'{sampling}%_{name}.csv')

# 3. 함수 실행 (함수 내부 로직은 고정)
perform_stratified_sampling(
    sample_fraction = sampling / 100,
    input_file = input_full_path, 
    output_file = output_full_path
)

📂 데이터 로딩 중...

✅ 샘플링 완료!
전체 데이터 수: 383,700 명
샘플링 데이터 수: 1,917 명
샘플 내 tier 비중:
tier
GOLD        24.83
SILVER      24.62
BRONZE      18.10
PLATINUM    17.63
EMERALD      9.29
DIAMOND      3.44
IRON         2.09
Name: proportion, dtype: float64%
💾 샘플 데이터가 D:\바탕화면\데이터 분석\프로젝트\04 최종 프로젝트\0.5%_tr1_puuids_iron_diamond.csv에 저장되었습니다.
